
# Docker + GitHub Container Registry: End-to-End Example (Plotting)

This repository demonstrates a **complete Docker workflow**:

1. create a Docker container locally (if possible)
2. upload source code and Dockerfile to GitHub
3. let GitHub Actions build the container
4. publish the container to GitHub Container Registry (GHCR)
5. download and run the container locally (private image, authenticated pull)

The example container generates a plot of

\[
\frac{\sin(x)}{(1+x^2)^{0.3}}
\]

and saves it as `03_plot_curve.png`.

---

## Repository structure

```

docker_apptainer_test/
├── 02_docker_hello/
│   └── README.md
├── 03_docker_plot/
│   ├── 03_plot_curve.py
│   ├── 03_dockerfile.txt
│   ├── requirements.txt
│   └── README.md
└── .github/
└── workflows/
└── docker-03-plot.yml

````

Generated files (plots, logs) are **not committed**.

---

## Step 1 — Python code executed in Docker

`03_docker_plot/03_plot_curve.py`

```python
import numpy as np
import matplotlib
matplotlib.use("Agg")  # non-interactive backend
import matplotlib.pyplot as plt

x = np.linspace(-20, 20, 2000)
y = np.sin(x) / (1.0 + x**2)**0.3

plt.figure(figsize=(6, 4))
plt.plot(x, y)
plt.xlabel("x")
plt.ylabel(r"$\sin(x)/(1+x^2)^{0.3}$")
plt.title("Test curve")
plt.tight_layout()
plt.savefig("03_plot_curve.png", dpi=150)
plt.close()

print("Saved 03_plot_curve.png")
````

Important:

* `Agg` backend is required for Docker / CI
* the script writes its output to the **current working directory**

---

## Step 2 — Python dependencies

`03_docker_plot/requirements.txt`

```
numpy
matplotlib
```

---

## Step 3 — Dockerfile

`03_docker_plot/03_dockerfile.txt`

```dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt /app/
RUN pip install --no-cache-dir -r requirements.txt

COPY 03_plot_curve.py /app/

CMD ["python", "03_plot_curve.py"]
```

Notes:

* the Docker image already acts as a **virtual environment**
* no `venv` activation is needed
* code lives in `/app` inside the image

---

## Step 4 — Local build and test (optional but recommended)

From the repository root:

```bash
docker build -t docker-plot-03 \
  -f 03_docker_plot/03_dockerfile.txt \
  03_docker_plot
```

Run locally:

```bash
docker run --rm \
  -v "$(pwd)/03_docker_plot:/app" \
  docker-plot-03
```

Result:

```
03_docker_plot/03_plot_curve.png
```

---

## Step 5 — GitHub Actions workflow (build on GitHub)

`.github/workflows/docker-03-plot.yml`

```yaml
name: Build and publish Docker image (03 plot)

on:
  push:
    paths:
      - "03_docker_plot/**"
      - ".github/workflows/docker-03-plot.yml"

jobs:
  build:
    runs-on: ubuntu-latest

    permissions:
      contents: read
      packages: write

    steps:
      - name: Checkout repository
        uses: actions/checkout@v4

      - name: Login to GitHub Container Registry
        run: |
          echo "${{ secrets.GITHUB_TOKEN }}" | \
          docker login ghcr.io -u ${{ github.actor }} --password-stdin

      - name: Build and push image
        run: |
          docker build \
            -t ghcr.io/eumetnet-e-ai/docker-plot-03:latest \
            -f 03_docker_plot/03_dockerfile.txt \
            03_docker_plot

          docker push ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

This workflow:

* runs on every push affecting `03_docker_plot`
* builds the Docker image
* publishes it to **GitHub Container Registry (GHCR)**

---

## Step 6 — Private container access: authentication required

The container image is **private**.
Pulling it locally requires authentication.

### Important concept

> Git authentication (SSH) is **not** container registry authentication.

---

## Step 7 — Create a GitHub Personal Access Token (PAT)

1. Open
   [https://github.com/settings/tokens](https://github.com/settings/tokens)
2. Choose **Tokens (classic)**
3. **Generate new token (classic)**

### Token settings

* **Note**: `ghcr-read-packages`
* **Scopes**:

  * ✅ `read:packages`
* All other scopes: **unchecked**

After creation:

* **Copy the token**
* **Authorize it for the organization** `eumetnet-e-ai` (SSO step)

---

## Step 8 — Login to GHCR locally

```bash
docker login ghcr.io
```

* **Username**: your GitHub username (e.g. `rolandpotthast`)
* **Password**: the PAT

Expected output:

```
Login Succeeded
```

---

## Step 9 — Download (pull) the container

```bash
docker pull ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

Verify:

```bash
docker images | grep docker-plot-03
```

---

## Step 10 — Run the downloaded container

### Correct run command (important!)

From the repository root:

```bash
docker run --rm \
  -v "$(pwd)/03_docker_plot:/app" \
  ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

Result:

```
03_docker_plot/03_plot_curve.png
```

### Why this matters

* `/app` inside the container contains the script
* the volume mount **must point to the directory with the code**
* mounting the wrong directory hides the application

---

## Platform warning (Apple Silicon / ARM)

You may see:

```
WARNING: image platform (linux/amd64) does not match host (linux/arm64)
```

This means:

* the image was built on x86_64 (GitHub runner)
* Docker is using transparent emulation

This is **expected and harmless** for this example.

To silence it explicitly:

```bash
docker run --rm \
  --platform linux/amd64 \
  -v "$(pwd)/03_docker_plot:/app" \
  ghcr.io/eumetnet-e-ai/docker-plot-03:latest
```

---

## Conceptual summary

* Dockerfile and source code live in **Git**
* Containers are built by **GitHub Actions**
* Images are stored in **GitHub Container Registry**
* Private images require **PAT authentication**
* Output files only persist with **volume mounts**

---

## Key takeaway

> Source code access, container access, and runtime execution are **separate concerns** — Docker and CI make this separation explicit.


